In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor, Booster
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error
import seaborn as sns

In [2]:
df = pd.read_pickle("logd_desc.gz")

In [3]:
df.shape

(1895249, 4)

In [4]:
df.head()

,smiles,name,logd,desc
0,Cc1cc(-n2ncc(=O)[nH]c2=O)ccc1C(=O)c1ccccc1Cl,1,2.69,"[2.047026309140364, 1053.1866841504302, 17.267..."
1,Cc1cc(-n2ncc(=O)[nH]c2=O)ccc1C(=O)c1ccc(C#N)cc1,2,1.82,"[1.988495947945752, 1118.5386170645465, 17.974..."
2,Cc1cc(-n2ncc(=O)[nH]c2=O)cc(C)c1C(O)c1ccc(Cl)cc1,3,2.64,"[2.072625090369594, 1015.4173196506215, 18.137..."
3,Cc1ccc(C(=O)c2ccc(-n3ncc(=O)[nH]c3=O)cc2)cc1,4,1.97,"[1.9257644721581226, 967.460585505423, 16.3969..."
4,Cc1cc(-n2ncc(=O)[nH]c2=O)ccc1C(=O)c1ccc(Cl)cc1,5,2.57,"[2.0074536470863955, 1035.9366841504304, 17.26..."


In [5]:
train, test = train_test_split(df)

In [6]:
%%time
X_train = np.stack(train.desc.values)
y_train = train.logd.values
X_test = np.stack(test.desc.values)
y_test = test.logd.values

CPU times: user 52.2 s, sys: 1min 16s, total: 2min 8s
Wall time: 3min 8s


In [7]:
%%time
lgbm = LGBMRegressor()
lgbm.fit(X_train,y_train)

CPU times: user 4min 4s, sys: 7.71 s, total: 4min 11s
Wall time: 21 s


LGBMRegressor()

In [8]:
%%time
pred = lgbm.predict(X_test)

CPU times: user 6.72 s, sys: 1.97 s, total: 8.69 s
Wall time: 727 ms


In [9]:
r2_score(y_test,pred)

0.8935354219587178

In [10]:
mean_squared_error(y_test,pred, squared=False)

0.8239139830945249

In [11]:
lgbm.booster_.save_model("model.txt")

In [12]:
mdl = Booster(model_file='model.txt')

In [13]:
pred = mdl.predict(X_test)

In [14]:
mean_squared_error(y_test,pred, squared=False)

0.8239139830945249

In [15]:
lgbm.booster_.save_model("logd.mdl")